In [1]:
#data from excel to databank
import pandas as pd
import sqlite3
import os
import sys
import re
import numpy as np
import datetime
import time
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
from matplotlib.ticker import MultipleLocator
from matplotlib.ticker import FormatStrFormatter
from matplotlib.ticker import AutoMinorLocator
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import AutoLocator
from matplotlib.ticker import ScalarFormatter
from matplotlib.ticker import FuncFormatter
from matplotlib.ticker import FixedLocator
from matplotlib.ticker import FixedFormatter
from matplotlib.ticker import StrMethodFormatter
from matplotlib.ticker import PercentFormatter
from matplotlib.ticker import EngFormatter

current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
file_path = os.path.join(parent_dir, 'Lets Meet DB Dump.xlsx')

print(file_path)
if os.path.exists(file_path):
    print("Datei gefunden. Laden...")
    df = pd.read_excel(file_path)
else:
    print("Datei nicht gefunden:", file_path)

ModuleNotFoundError: No module named 'pandas'

In [12]:
df.head()

,"Nachname, Vorname","Straße Nr, PLZ Ort",Telefon,Hobby1 %Prio1%; Hobby2 %Prio2%; Hobby3 %Prio3%; Hobby4 %Prio4%; Hobby5 %Prio5%;,E-Mail,Geschlecht (m/w/nonbinary),Interessiert an,Geburtsdatum
0,"Forster, Martin","Minslebener Str. 0, 46286, Dorsten",02372 8020,Fremdsprachenkenntnisse erweitern %78%; Im Was...,martin.forster@web.ork,m,w,07.03.1959
1,"Elina, Tsanaklidou","Gartenweg 13, 69126, Heidelberg",06221 / 98689,Jemanden massieren %57%; Mir die Probleme von ...,tsanaklidou.elina@1mal1.te,w,m,28.02.1958
2,"Vildan, şahin","Heinrich-Heine-Straße 99c, 17489, Greifswald",03834 / 22951,In der Stadt herumbummeln %94%; Mit Freunden z...,şahin.vildan@gmaiil.ork,w,m,27.07.1996
3,"Bäumker, Ellen","Weidenring 39, 81539, München",0162 / 249788,An einer Gruppenreise teilnehmen %97%; Seine f...,ellen.bäumker@web.ork,w,m,09.12.1992
4,"Bahadır, Bekiroğlu","Eichendorffstraße 20, 79379, Müllheim Baden",(07631) 67955,Ins Grüne fahren %23%; Holz hacken %67%; Fotos...,bekiroğlu.bahadır@autluuk.te,m,w,13.06.1983


In [13]:
# Aufteilen von "Nachname, Vorname" in separate Spalten
df[['Nachname', 'Vorname']] = df['Nachname, Vorname'].str.split(', ', expand=True)
# Aufteilen von "Straße Nr, PLZ Ort" in separate Spalten
df[['Straße Nr', 'PLZ', 'Ort']] = df['Straße Nr, PLZ Ort'].str.extract(r'^(.*?), (\d{5}), (.*)$')
# Extrahieren der Hobbys und Prioritäten
hobby_data = df["Hobby1 %Prio1%; Hobby2 %Prio2%; Hobby3 %Prio3%; Hobby4 %Prio4%; Hobby5 %Prio5%;"]
# Überprüfung auf leere oder fehlerhafte Hobby-Daten
def extract_hobbies(hobby_str):
    if pd.isna(hobby_str):  # Ignorieren von leeren Werten
        return {}
    hobbies = hobby_str.split(';')
    result = {}
    for i, hobby in enumerate(hobbies):
        match = re.match(r"^(.*?) %(\d+)%$", hobby.strip())
        if match:
            result[f'Hobby{i+1}'] = match.group(1).strip()
            result[f'Hobby{i+1}_Prio'] = int(match.group(2))
    return result

# Anwenden der Extraktion auf die Hobby-Daten
import re

hobby_details = hobby_data.apply(extract_hobbies)
hobby_df = pd.DataFrame(hobby_details.tolist())  # Umwandeln in DataFrame

# Zusammenführen mit dem ursprünglichen DataFrame
df = pd.concat([df, hobby_df], axis=1)

# Konvertieren des Geburtsdatums in ein datetime-Format
df['Geburtsdatum'] = pd.to_datetime(df['Geburtsdatum'], format='%d.%m.%Y')

# Entfernen der ursprünglichen kombinierten Spalten
df = df.drop(columns=[
    'Nachname, Vorname',
    'Straße Nr, PLZ Ort',
    'Hobby1 %Prio1%; Hobby2 %Prio2%; Hobby3 %Prio3%; Hobby4 %Prio4%; Hobby5 %Prio5%;'
])
df.head(5)

,Telefon,E-Mail,Geschlecht (m/w/nonbinary),Interessiert an,Geburtsdatum,Nachname,Vorname,Straße Nr,PLZ,Ort,Hobby1,Hobby1_Prio,Hobby2,Hobby2_Prio,Hobby3,Hobby3_Prio,Hobby4,Hobby4_Prio,Hobby5,Hobby5_Prio
0,02372 8020,martin.forster@web.ork,m,w,1959-03-07,Forster,Martin,Minslebener Str. 0,46286,Dorsten,Fremdsprachenkenntnisse erweitern,78.0,Im Wasser waten,80.0,Schwierige Probleme klären,61.0,Morgens Früh aufstehen,17.0,NaN,NaN
1,06221 / 98689,tsanaklidou.elina@1mal1.te,w,m,1958-02-28,Elina,Tsanaklidou,Gartenweg 13,69126,Heidelberg,Jemanden massieren,57.0,Mir die Probleme von anderen anhören,21.0,Abends seinem Partner Ereignisse des Tages erz...,53.0,Für einen guten Zweck spenden,25.0,NaN,NaN
2,03834 / 22951,şahin.vildan@gmaiil.ork,w,m,1996-07-27,Vildan,şahin,Heinrich-Heine-Straße 99c,17489,Greifswald,In der Stadt herumbummeln,94.0,Mit Freunden zusammen sein,20.0,Tagebuch schreiben,12.0,Ausschlafen,72.0,NaN,NaN
3,0162 / 249788,ellen.bäumker@web.ork,w,m,1992-12-09,Bäumker,Ellen,Weidenring 39,81539,München,An einer Gruppenreise teilnehmen,97.0,Seine finanziellen Angelegenheiten regeln,91.0,Etwas entwerfen oder zeichnen,93.0,NaN,NaN,NaN,NaN
4,(07631) 67955,bekiroğlu.bahadır@autluuk.te,m,w,1983-06-13,Bahadır,Bekiroğlu,Eichendorffstraße 20,79379,Müllheim Baden,Ins Grüne fahren,23.0,Holz hacken,67.0,Fotos oder Dias anschauen,84.0,"Lebensmittel einmachen, einfrieren, Vorräte an...",69.0,NaN,NaN
